In [ ]:
# Setting Up a Local Certificate Authority

# This notebook demonstrates how to set up a local Certificate Authority (CA) using OpenSSL,
# issue certificates, revoke them, and capture/decrypt traffic using the certificates.

# Note: This notebook uses bash commands. Make sure you have the bash kernel installed for Jupyter.
# You can install it using: pip install bash_kernel
# Then run: python -m bash_kernel.install

# 1. Set up the directory structure

mkdir -p ca/{root,intermediate}/{private,certs,newcerts,crl,csr}
mkdir -p ca/{root,intermediate}/index.txt
touch ca/{root,intermediate}/index.txt
echo 1000 > ca/root/serial
echo 1000 > ca/intermediate/serial

# 2. Create the root CA

# Generate the root key
openssl genrsa -aes256 -out ca/root/private/ca.key.pem 4096

# Generate the root certificate
openssl req -config openssl.cnf -key ca/root/private/ca.key.pem -new -x509 -days 7300 -sha256 -extensions v3_ca -out ca/root/certs/ca.cert.pem

# 3. Create the intermediate CA

# Generate the intermediate key
openssl genrsa -aes256 -out ca/intermediate/private/intermediate.key.pem 4096

# Generate the intermediate CSR
openssl req -config openssl.cnf -new -sha256 -key ca/intermediate/private/intermediate.key.pem -out ca/intermediate/csr/intermediate.csr.pem

# Sign the intermediate certificate with the root CA
openssl ca -config openssl.cnf -extensions v3_intermediate_ca -days 3650 -notext -md sha256 -in ca/intermediate/csr/intermediate.csr.pem -out ca/intermediate/certs/intermediate.cert.pem

# 4. Issue a server certificate

# Generate a server key
openssl genrsa -out ca/intermediate/private/www.example.com.key.pem 2048

# Generate a CSR for the server
openssl req -config openssl.cnf -key ca/intermediate/private/www.example.com.key.pem -new -sha256 -out ca/intermediate/csr/www.example.com.csr.pem

# Sign the server certificate with the intermediate CA
openssl ca -config openssl.cnf -extensions server_cert -days 375 -notext -md sha256 -in ca/intermediate/csr/www.example.com.csr.pem -out ca/intermediate/certs/www.example.com.cert.pem

# 5. Revoke a certificate

# Revoke the certificate
openssl ca -config openssl.cnf -revoke ca/intermediate/certs/www.example.com.cert.pem

# Generate a CRL
openssl ca -config openssl.cnf -gencrl -out ca/intermediate/crl/intermediate.crl.pem

# 6. Configure a client to use the certificates

# Combine the intermediate and root certificates
cat ca/intermediate/certs/intermediate.cert.pem ca/root/certs/ca.cert.pem > ca/intermediate/certs/ca-chain.cert.pem

# Install the CA certificates on the client (example for Ubuntu)
sudo cp ca/intermediate/certs/ca-chain.cert.pem /usr/local/share/ca-certificates/
sudo update-ca-certificates

# 7. Capture and decrypt HTTPS traffic (using Wireshark)

# Start capturing traffic
sudo tshark -i eth0 -w capture.pcap

# Decrypt the traffic (in Wireshark GUI)
# File -> Preferences -> Protocols -> TLS -> RSA keys list
# Add the server's private key: www.example.com.key.pem

# Note: The actual decryption is done in Wireshark GUI, not in this script.

# 8. Clean up (optional)

# rm -rf ca

# Note: Uncomment the line above to remove all generated files and directories.
# Be careful as this will delete all your CA files!